In [1]:
import tensorflow as tf
import tensorflow_federated as tff


ModuleNotFoundError: No module named 'tensorflow_federated'

In [ ]:
# Load the MNIST dataset
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()

# Preprocess the data
train_images = mnist_train[0] / 255.0
train_labels = mnist_train[1]
test_images = mnist_test[0] / 255.0
test_labels = mnist_test[1]

In [ ]:

# Create a simple neural network model
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  return model

In [ ]:
# Define the federated averaging process
federated_computation = tff.learning.build_federated_averaging_process(create_model)

# Initialize the global model
global_weights = federated_computation.initialize()


In [ ]:
# Run federated training
for round_num in range(10):
  # Sample clients
  client_data = tff.simulation.sample_clients(train_images, train_labels, num_clients=10)

  # Train models locally
  local_updates = tff.learning.build_local_training_process(create_model)()

  # Aggregate updates
  global_weights = federated_computation.next(global_weights, client_data, local_updates)

  # Evaluate the global model
  evaluation_metrics = tff.learning.metrics.accuracy(create_model(), global_weights, test_images, test_labels)
  print(f"Round {round_num}: Accuracy = {evaluation_metrics}")